In [ ]:
import healpy as hp
import numpy as np
import pickle
import h5py
import altair as alt
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
cd ..

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [ ]:
nside=512

In [ ]:
from astropy.table import QTable

In [ ]:
s4 = QTable.read("../202102_design_tool_run/instrument_model/cmbs4_instrument_model.tbl", format="ascii.ipac")

In [ ]:
s4 = s4[s4["telescope"] == "SAT"]

In [ ]:
s4.sort("center_frequency")

In [ ]:
s4

In [ ]:
channels = list(s4["band"])

In [ ]:
channels

In [ ]:
alt.data_transformers.enable("default")

In [ ]:
alt.data_transformers.disable_max_rows()

In [ ]:
import pysm3 as pysm
import so_pysm_models

from pysm3 import units as u

In [ ]:
import mapsims

In [ ]:
selection = {}
cl = {}
i_pol = 2

In [ ]:
from glob import glob

In [ ]:
toml_files = glob("*512.toml") + glob("cmb*.toml") + ["cib.toml", "ksz.toml", "tsz.toml"]

In [ ]:
toml_files 

In [ ]:
for config_file in toml_files: 
    mapsim = mapsims.from_config(["common.toml", config_file])
    component = config_file.split(".")[0].replace("_512", "")
    print(component)
    filename = f"output/{nside}/C_ell_{component}.pkl"
    with open(filename, "rb") as f:
        cl[component] = pickle.load(f)
    if mapsim.pysm_components_string is not None:
        component_objects = [so_pysm_models.get_so_models(mapsim.pysm_components_string, nside=nside)]
    else:
        component_objects = list(mapsim.pysm_custom_components.values())
    sky = pysm.Sky(nside=nside,
                   component_objects=component_objects,
                   output_unit=u.uK_CMB
                  )
    onecl = pd.DataFrame(
        {ch:cl[component][ch][i_pol] for ch in channels}
    ).reset_index().rename(columns={"index":"ell"})
    pysm_low = hp.anafast(
        sky.get_emission(s4["center_frequency"][0]),
    lmax=onecl.ell.max())[i_pol]
    onecl["PySM " + channels[0]] = pysm_low

    pysm_high = hp.anafast(
            sky.get_emission(
                s4[-1]["center_frequency"],
            ),
    lmax=onecl.ell.max())[i_pol]
    onecl["PySM " + channels[-1]] = pysm_high

    onecl = onecl[onecl.ell != 0]

    onecl_long = onecl.melt("ell", var_name="channel")
    onecl_long[onecl_long.value == 0] = np.nan
    selection[component] = alt.selection_multi(fields=['channel'], bind="legend")

    alt.Chart(onecl_long, title=component).mark_line().encode(
        x=alt.X("ell", scale=alt.Scale(type='log', base=10, domain=[1,1e4])),
        y=alt.Y("value", scale=alt.Scale(type='log', base=10)),
        color=alt.Color('channel', 
            sort=["PySM " + channels[0]] + list(channels) + ["PySM " + channels[-1]]
                       ),
        opacity=alt.condition(selection[component], alt.value(1), alt.value(0.2))
    ).add_selection(selection[component]).interactive().display()